### LIBRERÍAS

In [1]:
import pandas as pd
import numpy as np
import ast

### PROCESAMIENTO

In [2]:
df = pd.read_csv("Data/df_Clean_TripAdvisor 2023-09-05 14_24_13.csv")

df.columns

Index(['Unnamed: 0', 'Nombre', 'Score', 'Excelente', 'MuyBueno', 'Normal',
       'Malo', 'Pesimo', 'Num_Reviews', 'Posicion', 'Rango_precio',
       'Tipo_Cocina', 'Awards', 'Score_Comida', 'Score_Servicio',
       'Calidad_Precio', 'Ambiente', 'Direccion', 'Web_TripAdvisor', 'Latina',
       'Eur Este', 'Africana', 'China', 'Japonesa', 'Arabe', 'Mexicana',
       'Caribeña', 'Asiatica', 'America Norte', 'America Sur', 'Italiana',
       'Española', 'Eur Occidental', 'Asador', 'N/A', 'India',
       'Num_Reviews_Spain', 'Latitud', 'Longitud'],
      dtype='object')

In [4]:
# Columnas que nos interesan para el recomendador
df = df[["Nombre", "Rango_precio", "Tipo_Cocina"]]

In [6]:
# Función para fusionar conjuntos
def fusionar_conjuntos(cadena, conjunto_actual):
    try:
        # Utilizar ast.literal_eval para convertir la cadena en una lista
        lista = ast.literal_eval(cadena)
        # Fusionar la lista con el conjunto actual
        conjunto_actual.update(lista)
        return conjunto_actual
    
    except (SyntaxError, ValueError):
        # Manejar errores si la cadena no es válida
        return conjunto_actual

In [7]:
array_original = df.Tipo_Cocina.values
# Inicializar un conjunto vacío
conjunto_total = set()
# Aplicar la función de fusión a cada elemento del array
conjunto_total = np.vectorize(fusionar_conjuntos, otypes=[set])(array_original, conjunto_total)
# el conjunto total resultante
conjunto_total = conjunto_total[0]

In [8]:
# Crear columnas para cada tipo de cocina y rellenarlas con 1 y 0 si está presente en la columna tipo_cocina
for comida in conjunto_total:
    columna_comida = f"{comida}_presente"
    df[comida] = df['Tipo_Cocina'].apply(lambda x: 1 if comida in x else 0)

C:\Users\enaat\AppData\Local\Temp\ipykernel_8884\2443827351.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[comida] = df['Tipo_Cocina'].apply(lambda x: 1 if comida in x else 0)
C:\Users\enaat\AppData\Local\Temp\ipykernel_8884\2443827351.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[comida] = df['Tipo_Cocina'].apply(lambda x: 1 if comida in x else 0)
C:\Users\enaat\AppData\Local\Temp\ipykernel_8884\2443827351.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.in

In [9]:
# Aplicar dummies a la columna rango_precio
df_1 = pd.get_dummies(df['Rango_precio'])

df = pd.concat([df, df_1], axis=1)

In [10]:
# Verificar que las columnas creadas no sean boleanos
df["€"] = df["€"].astype(int)
df["€€ - €€€"] = df["€€ - €€€"].astype(int)
df["€€€€"] = df["€€€€"].astype(int)

In [11]:
df.drop("Rango_precio", axis=1, inplace=True)
df.drop("Tipo_Cocina", axis=1, inplace=True)

In [12]:
# Eliminar filas donde haya cadenas de restaurantes, queremos solo restaurantes únicos
conteo_columna = df['Nombre'].value_counts()

# Filtrar el DataFrame para excluir filas donde el conteo es mayor que 1
df = df[df['Nombre'].isin(conteo_columna[conteo_columna <= 1].index)]

In [13]:
# Comprobando duplicados
df.duplicated().sum()

0

In [ ]:
df.to_csv("Data/df_filtrado_madrid.csv")

### RESTAURANTES QUE PUEDE VALORAR EL USUARIO

In [14]:
df.Nombre.to_list()

['El Capote',
 'La Queda',
 'Limbo Tribunal',
 '400 Grados Pizza - Maria de Molina',
 "Naki's Urban Food",
 'Garden Navarra Castellana',
 'Restaurante Uval',
 'Healthy Poke Plaza Elíptica',
 'Trattoria da Piero',
 'La Fondue de Tell',
 'Sumeshi',
 'La Peineta',
 'BurgerMeister',
 'Mesón de la Guitarra',
 'Valdemeso',
 'El Ancla Del Lago',
 'Umami Restaurante',
 'Queen Burger Gourmet',
 'La Quintina',
 'Hawaii',
 '¡Que trabaje Rita',
 'Le Coq Las tablas',
 'Restaurante Abaco',
 'La Oficina Gastronomica',
 'Paellamar Tirso de Molina',
 'La Parrilla del Pinar, Madrid',
 'Boga Boga Taberna Premium',
 'Sana Vegetariana',
 'El Mono De La Pila',
 'El Nueve',
 'Taberna Los Madriles',
 'Artemisa Sol-Gran Via',
 'La Perdiz de Somontes',
 'Penalti Lounge Bar',
 'Marcial Bar',
 'La Usina',
 'Maki',
 'Ondinas Do Mendo',
 'Asador Lekeitio',
 'La Taberna de Mister Pinkleton',
 "L'Adoré  Café - Bistrot",
 'Brando',
 'El Respiro',
 'Café del Principe',
 'Comenssana',
 'Punch Room - The Madrid Edition',

### VALORACIÓN DEL USUARIO

In [15]:
datos = [
            ['El Capote', 6 ],
            ['BurgerMeister', 3 ],
            ['Casa Benito', 9],
            ["Le Coq Las tablas", 4]
        ]

df_usuario = pd.DataFrame(data = datos, columns = ["Nombre", "Puntuacion"])

### ALGORITMO RECOMENDADOR

In [16]:
# Concatenar los DataFrames basándose en la columna común
df_usuario = pd.merge(df, df_usuario, on='Nombre', how='left')

In [17]:
# Creando el df de pesos de los restaurantes que visitó el usuario
df_pesos = df_usuario[df_usuario["Puntuacion"] >0].copy()

In [18]:
# Asignandole valor a las caracteristicas de cada restaurante segun la puntuacion que dió el usuario
for columna in df_pesos.columns:
    if columna != "Nombre" and columna != "Puntuacion":
        df_pesos[columna] = df_pesos[columna]*df_pesos["Puntuacion"]

In [19]:
# Calculando el peso de cada característica
usuario_pesos = df_pesos.drop(["Nombre", "Puntuacion"], axis=1).sum()

usuario_pesos = usuario_pesos/usuario_pesos.sum()

In [20]:
# Creando df de restaurantes que no visitó el usuario
df_restaurantes = df_usuario[~(df_usuario["Puntuacion"] > 0)].copy()

In [21]:
# Asignandole el peso correspondientes a cada característica según el usuario de los demás restaurantes
weighted_movies_matrix = list()

for punto, restaurant in zip(usuario_pesos.values, df_restaurantes.iloc[:, 1:-1].values):
    weighted_movies_matrix.append(punto*restaurant)

weighted_movies_matrix = pd.DataFrame(data = weighted_movies_matrix, columns = df_restaurantes.drop(["Nombre", "Puntuacion"], axis=1).columns)

In [22]:
# Creando df de donde saldrá las recomendaciones
df_recomendador = pd.concat([df_restaurantes, weighted_movies_matrix.sum(axis = 1)], axis = 1)

df_recomendador.rename(columns= {0 : "Recomendacion"}, inplace=True)
# Ordenando los restaurantes de mayor peso a menor
df_recomendador.sort_values("Recomendacion", ascending = False, inplace=True)

In [23]:
# Filtrado de recomendaciones para el usuario
resultado = df_recomendador[df_recomendador["Recomendacion"] > 0]["Nombre"].to_list()

In [24]:
# De mejor recomendación a peor:
resultado

['Healthy Poke Peñagrande',
 'Candela Deli',
 'Jaleo’s',
 'Meson El Bodegon',
 'Sauvage Madrid',
 'Limbo Tribunal',
 'Ondinas Do Mendo',
 'La Peineta']